In [33]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
import nltk.tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

import re


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to C:\Users\Success
[nltk_data]     Corporate\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Success
[nltk_data]     Corporate\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Success
[nltk_data]     Corporate\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Success
[nltk_data]     Corporate\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Success
[nltk_data]     Corporate\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Success
[nltk_data]     Corpo

# Importing dataset

In [34]:
dataset = pd.read_csv("20k.csv",encoding='ISO-8859-1') #read csv + encoding
dataset.head() #checking table

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0
1,1,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0
2,2,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1
3,3,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0
4,4,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0


In [35]:
dataset.isnull().sum(axis=0) #checking if data cleaned
dataset.rename(columns={'Unnamed: 0':'id'}, inplace = True) #naming index column for easier visualisation

In [36]:
dataset['id'].replace('  ', np.nan, inplace=True) #cleaning columns with NaN or incorrect data
dataset= dataset.dropna(subset=['id'])  

# Cleaning dataset


In [37]:
dataset.dropna(how='any', inplace=True) 
#drop all NaN (double-cleaning)
#we are dropping 2000 ish values cos its less than 5% of our entire dataset - refer to 5% of missing values rule

In [38]:
dataset.id= pd.factorize(dataset.id)[0] #convert to int
dataset['title'] = dataset.title.astype(str) #convert to str
dataset['text'] = dataset.text.astype(str) #convert to str
dataset.label= pd.factorize(dataset.label)[0] #convert to int

# Making new columns


In [39]:
#creating new columns and converting dtyes again just to be sure

dataset['title_char_count'] = dataset['title'].str.len().astype(int) 
dataset['text_char_count'] = dataset['text'].str.len().astype(int)

dataset['title_char_count'] = dataset.title_char_count.astype(int) 

dataset['title_word_count'] = dataset['title'].str.split().str.len().astype(int)
dataset['text_word_count'] = dataset['text'].str.split().str.len().astype(int)

dataset.head()


,id,title,text,label,title_char_count,text_char_count,title_word_count,text_word_count
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0,119,3363,18,532
1,1,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0,133,165,20,22
2,2,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1,88,5679,12,786
3,3,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0,78,1381,11,202
4,4,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0,67,1131,10,154


# Information on dataset


In [40]:
dataset.shape
dataset.info()
dataset.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22012 entries, 0 to 22011
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                22012 non-null  int64 
 1   title             22012 non-null  object
 2   text              22012 non-null  object
 3   label             22012 non-null  int64 
 4   title_char_count  22012 non-null  int32 
 5   text_char_count   22012 non-null  int32 
 6   title_word_count  22012 non-null  int32 
 7   text_word_count   22012 non-null  int32 
dtypes: int32(4), int64(2), object(2)
memory usage: 1.0+ MB


id                   int64
title               object
text                object
label                int64
title_char_count     int32
text_char_count      int32
title_word_count     int32
text_word_count      int32
dtype: object

# Removal of Symbols


In [41]:
def remove_symbols(string):
    string = re.sub('[^a-zA-Z!?\']', ' ', string) #remove symbols
    return " ".join(string.split()) #eliminate white spaces

dataset['text'] = dataset['text'].apply(remove_symbols) #applying to text and title
dataset['title'] = dataset['title'].apply(remove_symbols)
dataset.head()

,id,title,text,label,title_char_count,text_char_count,title_word_count,text_word_count
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0,119,3363,18,532
1,1,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0,133,165,20,22
2,2,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1,88,5679,12,786
3,3,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0,78,1381,11,202
4,4,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0,67,1131,10,154


# Labelling of fake and real news


In [42]:
label_translated = np.where(dataset['label']==0, 'fake', 'real') #new columns
dataset.insert(loc = 4 ,column = 'label_translated',value = label_translated)

dataset.head()

,id,title,text,label,label_translated,title_char_count,text_char_count,title_word_count,text_word_count
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0,fake,119,3363,18,532
1,1,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0,fake,133,165,20,22
2,2,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1,real,88,5679,12,786
3,3,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0,fake,78,1381,11,202
4,4,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0,fake,67,1131,10,154


# Removal of Stopwords

In [43]:
from nltk.corpus import stopwords

def add_stopword_count_column(dataset, stop_words, text_column_name, stopword_count_column_name):
    index = 0
    for text in dataset[text_column_name]:
        textlist = text.split()
        stopword_number = 0
        for word in textlist:
            if word.lower() in stop_words:
                stopword_number += 1
        dataset.at[index, stopword_count_column_name] = stopword_number
        index += 1
    dataset[stopword_count_column_name] = dataset[stopword_count_column_name].fillna(0).astype(int)
    return dataset


stop_words = stopwords.words('english')

# removing stopwords from title
dataset['title'] = dataset['title'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))

# title's stopword count
dataset = add_stopword_count_column(dataset, stop_words, 'title', 'title_stopword_count')

# removing stopwords from text
dataset['text'] = dataset['text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))

# text's stopword count
dataset = add_stopword_count_column(dataset, stop_words, 'text', 'text_stopword_count')

dataset.head()


,id,title,text,label,label_translated,title_char_count,text_char_count,title_word_count,text_word_count,title_stopword_count,text_stopword_count
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0,fake,119,3363,18,532,6,48
1,1,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0,fake,133,165,20,22,6,1
2,2,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1,real,88,5679,12,786,0,46
3,3,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0,fake,78,1381,11,202,0,16
4,4,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0,fake,67,1131,10,154,1,10


In [44]:
dataset.dropna(how='any', inplace=True) #drop na values and setting dtypes of new columns

dataset['title_stopword_count'] = dataset.title_stopword_count.astype(int)
dataset['text_stopword_count'] = dataset.text_stopword_count.astype(int)


### Lemmatisation

In [45]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
    
# lemmatisation of text
dataset['text_lemmatized'] = dataset['text'].apply(lambda x: " ".join([lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) 
                                                        for word, pos_tag in nltk.pos_tag(x.split())]))
# lemmatisation of title
dataset['title_lemmatized'] = dataset['title'].apply(lambda x: " ".join([lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) 
                                                          for word, pos_tag in nltk.pos_tag(x.split())]))


In [46]:
dataset.head()

,id,title,text,label,label_translated,title_char_count,text_char_count,title_word_count,text_word_count,title_stopword_count,text_stopword_count,text_lemmatized,title_lemmatized
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0,fake,119,3363,18,532,6,48,No comment expect Barack Obama Members FYF Fuk...,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1,1,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0,fake,133,165,20,22,6,1,Now demonstrator gather last night exercise co...,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...
2,2,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1,real,88,5679,12,786,0,46,A dozen politically active pastor come private...,Bobby Jindal raise Hindu use story Christian c...
3,3,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0,fake,78,1381,11,202,0,16,The RS Sarmat missile dub Satan replace SS Fli...,SATAN Russia unvelis image terrify new SUPERNU...
4,4,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0,fake,67,1131,10,154,1,10,All say one time someone sue Southern Poverty ...,About Time! Christian Group Sues Amazon SPLC D...


# Sentiment Generation - polarity 


In [47]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia

sia = sia()

def get_polarity(text):
    #without limiting text length, code took too long to run 
    text = text[:text.find(' ', 30)] if len(text) > 30 else text 
    text = " ".join(text.split()[:30])

    polarity_scores = sia.polarity_scores(text)
    polarity, score = sorted(polarity_scores.items(), key=lambda x: x[1], reverse=True)[:1][0]
    if polarity == 'pos': #improving dataset visualisation
        polarity = 'positive'
    elif polarity == 'neg':
        polarity = 'negative'
    elif polarity == 'neu':
        polarity = 'neutral'
    else:
        polarity = 'compound';
    return polarity, score

polarity_score = dataset['title'].apply(get_polarity)
dataset['title_polarity'] = polarity_score.str[0]
dataset['title_polarity_score'] = polarity_score.str[1]

polarity_score = dataset['text'].apply(get_polarity)
dataset['text_polarity'] = polarity_score.str[0]
dataset['text_polarity_score'] = polarity_score.str[1]


dataset.head()


,id,title,text,label,label_translated,title_char_count,text_char_count,title_word_count,text_word_count,title_stopword_count,text_stopword_count,text_lemmatized,title_lemmatized,title_polarity,title_polarity_score,text_polarity,text_polarity_score
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0,fake,119,3363,18,532,6,48,No comment expect Barack Obama Members FYF Fuk...,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,neutral,0.630,neutral,0.645
1,1,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0,fake,133,165,20,22,6,1,Now demonstrator gather last night exercise co...,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,positive,0.511,neutral,1.000
2,2,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1,real,88,5679,12,786,0,46,A dozen politically active pastor come private...,Bobby Jindal raise Hindu use story Christian c...,neutral,1.000,neutral,0.526
3,3,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0,fake,78,1381,11,202,0,16,The RS Sarmat missile dub Satan replace SS Fli...,SATAN Russia unvelis image terrify new SUPERNU...,neutral,0.519,neutral,1.000
4,4,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0,fake,67,1131,10,154,1,10,All say one time someone sue Southern Poverty ...,About Time! Christian Group Sues Amazon SPLC D...,neutral,1.000,neutral,1.000


# Emotion Generation

In [48]:
!pip install text2emotion
import text2emotion as t2e

def get_emotion(sentence):
    #important as code took too long to run without limiting
    sentence = sentence[:sentence.find(' ', 30)] if len(sentence) > 30 else sentence
    sentence = " ".join(sentence.split()[:30])
  
    emotion_scores = t2e.get_emotion(sentence)
    if not emotion_scores:
        return "neutral"
    emotion, score = max(emotion_scores.items(), key=lambda x: x[1])
    if score == 0.0:
        return "neutral"
    else:
        return emotion

    
dataset['title_emotion'] = dataset['title'].apply(get_emotion)
dataset['text_emotion'] = dataset['text'].apply(get_emotion)

dataset.head()


,id,title,text,label,label_translated,title_char_count,text_char_count,title_word_count,text_word_count,title_stopword_count,text_stopword_count,text_lemmatized,title_lemmatized,title_polarity,title_polarity_score,text_polarity,text_polarity_score,title_emotion,text_emotion
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0,fake,119,3363,18,532,6,48,No comment expect Barack Obama Members FYF Fuk...,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,neutral,0.630,neutral,0.645,Surprise,Happy
1,1,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0,fake,133,165,20,22,6,1,Now demonstrator gather last night exercise co...,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,positive,0.511,neutral,1.000,neutral,neutral
2,2,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1,real,88,5679,12,786,0,46,A dozen politically active pastor come private...,Bobby Jindal raise Hindu use story Christian c...,neutral,1.000,neutral,0.526,neutral,neutral
3,3,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0,fake,78,1381,11,202,0,16,The RS Sarmat missile dub Satan replace SS Fli...,SATAN Russia unvelis image terrify new SUPERNU...,neutral,0.519,neutral,1.000,neutral,neutral
4,4,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0,fake,67,1131,10,154,1,10,All say one time someone sue Southern Poverty ...,About Time! Christian Group Sues Amazon SPLC D...,neutral,1.000,neutral,1.000,Surprise,neutral


# Deleting empty rows after cleaning

In [49]:
empty_string_counts = (dataset == '').sum()
for col, count in empty_string_counts.iteritems():
    print(f"{col}:\t\t\t{count}")

dataset.replace('', np.nan, inplace=True) #final cleaning

id:			0
title:			0
text:			0
label:			0
label_translated:			0
title_char_count:			0
text_char_count:			0
title_word_count:			0
text_word_count:			0
title_stopword_count:			0
text_stopword_count:			0
text_lemmatized:			0
title_lemmatized:			0
title_polarity:			0
title_polarity_score:			0
text_polarity:			0
text_polarity_score:			0
title_emotion:			0
text_emotion:			0


In [50]:
dataset.isnull().sum(axis=0)

id                      0
title                   0
text                    0
label                   0
label_translated        0
title_char_count        0
text_char_count         0
title_word_count        0
text_word_count         0
title_stopword_count    0
text_stopword_count     0
text_lemmatized         0
title_lemmatized        0
title_polarity          0
title_polarity_score    0
text_polarity           0
text_polarity_score     0
title_emotion           0
text_emotion            0
dtype: int64

In [51]:
dataset.dropna(inplace=True)
dataset.reset_index(inplace=True, drop=True)
dataset.index.name = 'id'


dataset.shape
dataset.head()

,id,title,text,label,label_translated,title_char_count,text_char_count,title_word_count,text_word_count,title_stopword_count,text_stopword_count,text_lemmatized,title_lemmatized,title_polarity,title_polarity_score,text_polarity,text_polarity_score,title_emotion,text_emotion
id,,,,,,,,,,,,,,,,,,,
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0,fake,119,3363,18,532,6,48,No comment expect Barack Obama Members FYF Fuk...,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,neutral,0.630,neutral,0.645,Surprise,Happy
1,1,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0,fake,133,165,20,22,6,1,Now demonstrator gather last night exercise co...,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,positive,0.511,neutral,1.000,neutral,neutral
2,2,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1,real,88,5679,12,786,0,46,A dozen politically active pastor come private...,Bobby Jindal raise Hindu use story Christian c...,neutral,1.000,neutral,0.526,neutral,neutral
3,3,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0,fake,78,1381,11,202,0,16,The RS Sarmat missile dub Satan replace SS Fli...,SATAN Russia unvelis image terrify new SUPERNU...,neutral,0.519,neutral,1.000,neutral,neutral
4,4,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0,fake,67,1131,10,154,1,10,All say one time someone sue Southern Poverty ...,About Time! Christian Group Sues Amazon SPLC D...,neutral,1.000,neutral,1.000,Surprise,neutral


In [52]:
dataset.to_csv("20k_cleaned.csv", sep=',', encoding='utf-8') #getting cleaned dataset